In [15]:
import numpy as np 
import pandas as pd 

In [16]:
import torch
from tqdm.notebook import tqdm

In [17]:
folder = 'data_yang_dipake/'

In [18]:
df = pd.read_csv(f'{folder}dataML.csv')

In [19]:
df.head()

,Unnamed: 0,full_text,Label
0,0,@Iam_Nobody_1145 Sama aja dong kek kasus kudat...,2.0
1,1,@Bambangmulyonoo @PDI_Perjuangan Salah satu ya...,2.0
2,2,@Miduk17 @agussusilo7562 @ganjarpranowo @mohma...,2.0
3,3,@Paltiwest Pdi p jgn mw menampung mental2 pngk...,2.0
4,4,@Minietweets_ @adri_7i @adi_sareang @alextham8...,0.0


In [20]:
df = df.drop(columns='Unnamed: 0')

In [21]:
df['full_text'] = df['full_text'].astype(str)

In [22]:
df = df.dropna()

In [23]:
df['Label'] = df['Label'].astype(int)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3430 entries, 0 to 3432
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   full_text  3430 non-null   object
 1   Label      3430 non-null   int32 
dtypes: int32(1), object(1)
memory usage: 67.0+ KB


In [25]:
import re

# Translate emoticon
emoticon_data_path = 'external/emoticon.txt'
emoticon_df = pd.read_csv(emoticon_data_path, sep='\t', header=None)
emoticon_dict = dict(zip(emoticon_df[0], emoticon_df[1]))

def translate_emoticon(t):
    for w, v in emoticon_dict.items():
        pattern = re.compile(re.escape(w))
        match = re.search(pattern,t)
        if match:
            t = re.sub(pattern,v,t)
    return t

In [26]:
def remove_newline(text):
    return re.sub('\n', ' ',text)

In [27]:
def remove_url(text):
    return re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', text)

In [28]:
def remove_excessive_whitespace(text):
    return re.sub('  +', ' ', text)

In [29]:
def remove_non_alphabet(text):
    output = re.sub('[^a-zA-Z ]+', '', text)
    return output

In [30]:
def remove_twitter_formatting(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)
    text = re.sub(r'\brt\b', '', text)
    return text

In [31]:
import itertools

def remove_repeating_characters(text):
    return ''.join(''.join(s)[:1] for _, s in itertools.groupby(text))

In [32]:
def preprocess_text(text):
    transformed_text = text.lower()
    transformed_text = remove_newline(text)
    transformed_text = remove_url(transformed_text)
    transformed_text = remove_twitter_formatting(transformed_text)
    transformed_text = translate_emoticon(transformed_text)
    transformed_text = transformed_text.lower()
    transformed_text = remove_repeating_characters(transformed_text)
    transformed_text = remove_non_alphabet(transformed_text)
    transformed_text = remove_excessive_whitespace(transformed_text)
    transformed_text = transformed_text.lower().strip()
    return transformed_text

In [33]:
df['preprocess'] = df['full_text'].apply(preprocess_text)

In [34]:
label_map = {0: 'Positive', 1: 'Toxic', 2: 'Critique', 3: 'Others'}

In [35]:
df['Sentiment'] = df['Label'].replace(label_map)

In [36]:
df.head()

,full_text,Label,preprocess,Sentiment
0,@Iam_Nobody_1145 Sama aja dong kek kasus kudat...,2,nobody sama aja dong kek kasus kudatuli yg ga ...,Critique
1,@Bambangmulyonoo @PDI_Perjuangan Salah satu ya...,2,perjuangan salah satu yang kencang teriak peri...,Critique
2,@Miduk17 @agussusilo7562 @ganjarpranowo @mohma...,2,tp pdi ga berani ato jd penghalang yah,Critique
3,@Paltiwest Pdi p jgn mw menampung mental2 pngk...,2,pdi p jgn mw menampung mental pngkhianat jk pl...,Critique
4,@Minietweets_ @adri_7i @adi_sareang @alextham8...,0,i sareang back pdip dh pdi kan udah biasa dibe...,Positive


In [37]:
df.Sentiment.value_counts()

Sentiment
Toxic       1303
Others       950
Positive     834
Critique     343
Name: count, dtype: int64

In [38]:
possible_labels = df.Sentiment.unique()

In [39]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [40]:
label_dict

{'Critique': 0, 'Positive': 1, 'Toxic': 2, 'Others': 3}

In [41]:
df.Sentiment = df['Sentiment'].map(label_dict)

In [42]:
df = df.drop(columns='full_text')

In [43]:
from sklearn.model_selection import train_test_split

In [44]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.Sentiment.values, 
                                                  test_size=0.3, 
                                                  random_state=42,
                                                  stratify=df.Sentiment.values)

In [45]:
df['data_type'] = ['not_set']*df.shape[0]

In [46]:
df.head()

,Label,preprocess,Sentiment,data_type
0,2,nobody sama aja dong kek kasus kudatuli yg ga ...,0,not_set
1,2,perjuangan salah satu yang kencang teriak peri...,0,not_set
2,2,tp pdi ga berani ato jd penghalang yah,0,not_set
3,2,pdi p jgn mw menampung mental pngkhianat jk pl...,0,not_set
4,0,i sareang back pdip dh pdi kan udah biasa dibe...,1,not_set


In [47]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [48]:
df.groupby(['Sentiment', 'data_type']).count()

Label  preprocess
Sentiment data_type                   
0         train        240         240
          val          103         103
1         train        584         584
          val          250         250
2         train        912         912
          val          391         391
3         train        665         665
          val          285         285

In [49]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [50]:
tokenizer = BertTokenizer.from_pretrained(
    'indolem/indobertweet-base-uncased',
    do_lower_case=True
)

In [51]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].preprocess.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].preprocess.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].Sentiment.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].Sentiment.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Python310\lib\site-packages\transformers\tokenization_utils_base.py:2618: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [52]:
dataset_train = TensorDataset(input_ids_train, 
                              attention_masks_train,
                              labels_train)

dataset_val = TensorDataset(input_ids_val, 
                            attention_masks_val,
                           labels_val)

In [53]:
len(dataset_train)

2401

In [54]:
dataset_val.tensors

(tensor([[    3,  3961,  1911,  ...,     0,     0,     0],
         [    3,  3594,  3583,  ...,     0,     0,     0],
         [    3, 30350, 26971,  ...,     0,     0,     0],
         ...,
         [    3,  5014,  1801,  ...,     0,     0,     0],
         [    3,  2462,  1995,  ...,     0,     0,     0],
         [    3,  2755, 29953,  ...,     0,     0,     0]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 tensor([0, 0, 1,  ..., 3, 1, 0]))

In [55]:
from transformers import BertForSequenceClassification

In [56]:
model = BertForSequenceClassification.from_pretrained(
                                      'indolem/indobertweet-base-uncased', 
                                      num_labels = len(label_dict),
                                      output_attentions = False,
                                      output_hidden_states = False
                                     )

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indolem/indobertweet-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [57]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [58]:
batch_size = 4

dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=32
)

In [59]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [60]:
optimizer = AdamW(
    model.parameters(),
    lr = 1e-5,
    eps = 1e-8
)

c:\Python310\lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [61]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps = len(dataloader_train)*epochs
)

In [62]:
from sklearn.metrics import f1_score

In [63]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average = 'weighted')

In [64]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [65]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [66]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cuda


In [67]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [68]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    loss_train_total = 0
    
    progress_bar = tqdm(dataloader_train, 
                        desc='Epoch {:1d}'.format(epoch), 
                        leave=False, 
                        disable=False)
    
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }
        
        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total +=loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})     
    
    #torch.save(model.state_dict(), f'Models/BERT_ft_Epoch{epoch}.model')
    
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted): {val_f1}')

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/601 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.8756859953113879


  0%|          | 0/33 [00:00<?, ?it/s]

Validation loss: 0.7280180725184354
F1 Score (weighted): 0.6980129071757599


...

Validation loss: 1.7941643433137373
F1 Score (weighted): 0.7199381718865804


Epoch 10:   0%|          | 0/601 [00:00<?, ?it/s]


Epoch 10
Training loss: 0.13707148038710315


  0%|          | 0/33 [00:00<?, ?it/s]

Validation loss: 1.80549038178993
F1 Score (weighted): 0.7090211673585279


In [70]:
accuracy_per_class(predictions, true_vals)

Class: Critique
Accuracy:28/103

Class: Positive
Accuracy:175/250

Class: Toxic
Accuracy:285/391

Class: Others
Accuracy:243/285

